In [13]:
import credentials as cr
import SQL_requests as rq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import mlflow
from db_connect import Db_connect
from recommendation_system import Recommendation

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
data_path = cr.DATA_PATH

db = Db_connect()
rc = Recommendation()

mlflow.sklearn.autolog()

In [15]:
db.init_connection()
top_30_titles = pd.read_sql_query(rq.SELECT_PLAYLIST_TITLES + "'6AsIwsN6mH7dQSChLsqSlY'", db.connector).set_index('id')
track_titles = pd.read_sql_query(rq.SELECT_TRACKS_TITLES, db.connector).set_index('id')
track_features = pd.read_sql_query(rq.SELECT_TRACKS_FEATURES, db.connector).set_index('id')
db.close_connection()

## Process data & data split

In [1]:
track_features_processed = rc.process_data(track_features)
playlist, nonplaylist = rc.generate_playlist_nonplaylist(track_features_processed, top_30_titles)

train_test_dataset = track_features_processed.copy()
train_test_dataset['playlist'] = train_test_dataset.apply(lambda x: int(x.name in top_30_titles.index), axis=1)

classifier_data = train_test_dataset.copy()
classifier_data = classifier_data.sort_values(['playlist'], ascending=False).head(100)

classifier_target = classifier_data['playlist']

classifier_data = classifier_data.drop('loudness',axis = 1)
classifier_data = classifier_data.drop('playlist',axis = 1)

X_train, X_test, y_train, y_test = train_test_split(classifier_data, classifier_target, test_size=0.3, random_state=42)

NameError: name 'rc' is not defined

In [12]:
# matrix = train_test_dataset[['danceability', 'acousticness', 'loudness', 'energy', 'speechiness', 'instrumentalness', 'valence', 'liveness', 'tempo', 'playlist']].corr()
# sn.heatmap(matrix, annot=True)
# plt.show()

## Starting point

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

## Finding the right K

In [ ]:
error_rate_knn = []
accuracy_knn = []

for i in range(1,30):

    # kNN
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    y_pred = knn.predict(X_test)

    error_rate_knn.append(np.mean(y_pred != y_test))
    accuracy_knn.append(metrics.accuracy_score(y_test, y_pred))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6))
fig.suptitle('Model evolution depending on k')

ax1.plot(range(1,30),error_rate_knn,color='red')
ax1.set_title('Error Rate vs. K Value')
ax1.set_xlabel('K')
ax1.set_ylabel('Error Rate')

ax2.plot(range(1,30),accuracy_knn,color='green')
ax2.set_title('Accuracy vs. K Value')
ax2.set_xlabel('K')
ax2.set_ylabel('Accuracy')

plt.show()

## Write the algo

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4, algorithm='auto', weights='distance', metric='minkowski')

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

## Testing multi metrics

In [ ]:
# todo

## Testing weights

In [ ]:
# todo

## Random PCA / Grid Search

## Ajout année

## Recommandation finale

In [ ]:
recommendation = nonplaylist.copy()
recommendation = recommendation.drop('loudness',axis = 1)

recommendation['predict'] = recommendation.apply(lambda x: knn.predict([x])[0], axis=1)
recommendation['proba'] = recommendation.loc[:, recommendation.columns != 'predict'].apply(lambda x: knn.predict_proba([x])[0].max(), axis=1)

recommendation = recommendation[recommendation['predict']==1]
recommendation = recommendation.sort_values('proba', ascending=False).head(10)

recommendation = recommendation.merge(track_titles, left_index=True, right_index=True)
rc.visualize_cover(recommendation)
display(recommendation)